In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col, count, when
import re  # Para expressões regulares e tratamento de colunas

# Inicializa uma sessão Spark
spark = SparkSession.builder \
    .appName("Balance Sheet Analysis") \
    .getOrCreate()

# Carrega o arquivo Excel usando Pandas
excel_file = '/content/drive/MyDrive/Colab Notebooks/BIG_DATA/Projeto_Big_Data_Contabil/planilha-balanco-patrimonial-contaazul (1).xlsx'
excel_data = pd.read_excel(excel_file)

# Converte o Pandas DataFrame para um PySpark DataFrame
df = spark.createDataFrame(excel_data)

# Função para normalizar os nomes das colunas (remover acentos, espaços, etc.)
def normalize_column_name(col_name):
    col_name = col_name.lower()  # Converte para minúsculas
    col_name = re.sub(r'[áàãâ]', 'a', col_name)  # Remove acentos
    col_name = re.sub(r'[éê]', 'e', col_name)
    col_name = re.sub(r'[í]', 'i', col_name)
    col_name = re.sub(r'[óôõ]', 'o', col_name)
    col_name = re.sub(r'[ú]', 'u', col_name)
    col_name = re.sub(r'[ç]', 'c', col_name)
    col_name = re.sub(r'[^a-z0-9_]', '_', col_name)  # Substitui caracteres especiais por sublinhados
    return col_name

# Renomeia as colunas no PySpark DataFrame
for old_col_name in df.columns:
    new_col_name = normalize_column_name(old_col_name)
    df = df.withColumnRenamed(old_col_name, new_col_name)

# Imprime os nomes das colunas para depuração
print("Nomes das colunas:", df.columns)

# Exibe o DataFrame para inspecionar os dados e os nomes das colunas
df.show() # Adicione esta linha para visualizar o DataFrame

# Verifica se há valores nulos em cada coluna
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# Remove linhas duplicadas (se houver)
df = df.dropDuplicates()

# Substitui valores nulos por 0 em todas as colunas
df = df.fillna(0)

# Exemplo de cálculo da Liquidez Corrente (Ativo Circulante / Passivo Circulante)
# Substitua 'unnamed__1' e 'unnamed__2' pelos nomes corretos das colunas após a normalização
# Certifique-se de usar os nomes corretos após a normalização.

# Inspecione a saída de df.show() acima para identificar os nomes corretos das colunas
# que representam 'Ativo Circulante' e 'Passivo Circulante' após a normalização.
# Por exemplo, se as colunas foram renomeadas para 'ativo_circulante' e 'passivo_circulante', 
# use esses nomes no código abaixo:

df = df.withColumn("liquidez_corrente", col("unnamed__1") / col("unnamed__2"))  # Substitua pelos nomes corretos

# Exemplo de cálculo do Índice de Endividamento (Passivo Total / Patrimônio Líquido)
# Similarmente, substitua 'unnamed__3' e 'unnamed__4' pelos nomes corretos das colunas 
# que representam 'Passivo Total' e 'Patrimônio Líquido' após a normalização.
df = df.withColumn("endividamento", col("unnamed__3") / col("unnamed__4"))  # Substitua pelos nomes corretos


# Mostra os resultados dos cálculos
df.select("liquidez_corrente", "endividamento").show()

# Converte o PySpark DataFrame de volta para Pandas
df_pandas = df.toPandas()

# Salva como um novo arquivo Excel
output_file = 'balanco_patrimonial_processado.xlsx'
df_pandas